In [ ]:
import mbuild as mb
from foyer import Forcefield
from mbuild.lib.recipes.polymer import Polymer
import unyt as u

import mosdef_cassandra as mc

## Prepare recipes

### Exercise 1

In [ ]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        # Create a partilce name _CH2
        _ch2 = mb.Compound(name="_CH2")
        self.add(_ch2, "_CH2")
        
        # Create ports to be attached to the _CH2 particles
        port_up = mb.Port(anchor=_ch2, orientation=[0, 0, 1], separation=0.07)
        port_down = mb.Port(anchor=_ch2, orientation=[0, 0, -1], separation=0.07)
        self.add(port_up, "up")
        self.add(port_down, "down")
        
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        # Create a particle name _CH3
        _ch3 = mb.Compound(name="_CH3")
        self.add(_ch3, "_CH3")
        
        # Create a port to be attached to the _CH3 particle
        port_up = mb.Port(anchor=_ch3, orientation=[0, 0, 1], separation=0.07)
        self.add(port_up, "up")

            

### Exercise 2 

In [ ]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            _ch4 = mb.Compound(name="_CH4")
            self.add(_ch4, "_CH4")
        elif n == 2:
            _ch3a = _CH3() 
            _ch3b = _CH3() 
            self.add(_ch3a, "_CH3_a")
            self.add(_ch3b, "_CH3_b")
            mb.force_overlap(move_this=_ch3a, 
                             from_positions=_ch3a["up"], 
                             to_positions=_ch3b["up"])
        else:
            _ch3top = _CH3()
            self.add(_ch3top, "link0")
            for i in range(n - 2):
                _ch2 = _CH2()
                mb.force_overlap(move_this=_ch2, 
                                 from_positions=_ch2["down"],
                                 to_positions=self[f"link{i}"]["up"])
                self.add(_ch2, f"link{i+1}")
                
            _ch3bot = _CH3()
            self.add(_ch3bot, f"link{i+2}")
            mb.force_overlap(move_this=_ch3bot, 
                             from_positions=_ch3bot["up"], 
                             to_positions=self[f"link{i+1}"]["up"])


In [ ]:
from mbuild.lib.recipes import Polymer

cg_alkane = Polymer(monomers=[_CH2()], end_groups=[_CH3(), _CH3()])
cg_alkane.build(n=5)

### Exercise 3

In [ ]:
n_pentane = CG_Alkane(5)
pentane_box = mb.fill_box(compound=n_pentane, 
                          n_compounds=100, 
                          box=[3.2, 3.2, 3.2],
                          seed=12345, 
                          edge=0.2)

### Exercise 4

In [ ]:
import foyer 
from foyer import Forcefield

trappe = Forcefield(name="trappe-ua")
pentane_box.energy_minimize(forcefield="trappe-ua", 
                            steps=10**6)

pentane_box.visualize()


### Exercise 5a - Simulation with MoSDeF Cassandra

In [ ]:
%cd mc_sim/
# Load forcefields
trappe = Forcefield(name="trappe-ua")

# Use foyer to apply forcefields
typed_pentane = trappe.apply(n_pentane)
species_list = [typed_pentane]

# Create box and species list
box_list = [pentane_box]
mols_in_boxes = [[100]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moveset = mc.MoveSet("npt", species_list)

moveset.prob_volume = 0.010

thermo_props = [
    "energy_total",
    "energy_intervdw",
    "pressure",
    "volume",
    "nmols",
    "mass_density",
]

custom_args = {
    "run_name": "equil",
    "charge_style": "none",
    "rcut_min": 2.0 * u.angstrom,
    "vdw_cutoff": 14.0 * u.angstrom,
    "units": "sweeps",
    "steps_per_sweep": 450,
    "coord_freq": 50,
    "prop_freq": 10,
    "properties": thermo_props,
}


mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=2400,
    pressure=14.02 * u.bar,
    temperature=372 * u.K,
    **custom_args,
)
%cd ..

### Exercise 5b - Simulation with GROMACS

In [ ]:
%cd ..

In [ ]:
pentane_box.save("gmx_sim/init.gro", overwrite=True)
pentane_box.save("gmx_sim/init.top", forcefield_name="trappe-ua", overwrite=True)

In [ ]:
em_mdp = """
integrator          = steep
nsteps              = 500000
emstep              = 0.002
emtol               = 10
dt                  = 0.002

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

gen_vel             = yes
gen-temp            = 372.0
gen-seed            = 4

tcoupl              = no

pcoupl              = no

pbc                 = xyz

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

nvt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1
ref_t               = 372.0

pcoupl              = no

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

npt_mdp = """
integrator          = md
nsteps              = 1000000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 1000
nstvout             = 1000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz 

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4 

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4 

gen_vel             = no

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1 
ref_t               = 372.0

pcoupl                   = parrinello-rahman
pcoupltype               = isotropic
nstpcouple               = -1
tau-p                    = 10.0
compressibility          = 4.5e-5
ref-p                    = 14.02

DispCorr            = EnerPres

constraint-algorithm = LINCS
constraints         = all-bonds
"""

with open("gmx_sim/em.mdp", "w") as f:
    f.write(em_mdp)

with open("gmx_sim/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

with open("gmx_sim/npt.mdp", "w") as f:
    f.write(npt_mdp)

In [ ]:
%cd gmx_sim
!gmx grompp -f em.mdp -o em.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm em -s em.tpr -cpi em.cpt

!gmx grompp -f nvt.mdp -o nvt.tpr -c em.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt

!gmx grompp -f npt.mdp -o npt.tpr -c nvt.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm npt -s npt.tpr -cpi npt.cpt
%cd ..

### Exercise 6

In [ ]:
import numpy as np
import pylab as plt

data = np.genfromtxt("equil.out.prp", skip_header=3)

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r'MC sweeps')
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

ax.plot(data[:, 0], data[:,-1], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()

In [ ]:
import numpy as np
import pylab as plt 

import panedr
from panedr import edr_to_df

data = edr_to_df("gmx_sim/npt.edr")

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r"MD Step")
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

dt, density = list(), list()
for i, j in enumerate(data["Density"]):
    dt.append(i)
    density.append(j)
    
ax.plot(dt, density, "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()